In [1]:
# Required package installations.
! pip install pandas
! pip install petl

In [1]:
# Required imports.
import pandas as pd
import petl as etl
import glob
import os

In [2]:
# Set up variables containing location of relevant csv and json.
globbed_csv_files = glob.glob("data/*.csv")
globbed_json_files = glob.glob("data/*.json")

# Make headers in branch data consistent.
# The structure of the tables are uniform, therefore only heading names need to be adjusted.
headers = ['year', 'month', 'day', 'hour', 'product', 'quantity', 'amount_in_gbp']

In [4]:
# Read all relevant branch csv files and add a new column called 'branch_name'.
# Clean and convert the 'quantity' field from an object to int.
csv_branch = [] 
for i in globbed_csv_files:

    csv_data = pd.read_csv(i, header=0, names=headers, low_memory=False)
    extracted_filename = os.path.basename(i[15:-4]) # removes '20xx-20xx_' and '.csv' from the filename
    removed_characters = extracted_filename.replace('_',' ') # remove '_' between the branch terms
    csv_data['branch_name'] = removed_characters # 'branch_name' is consistent with other files

    remove_non_value = csv_data['quantity'].replace('-','0') # remove any '-' from quantity field; not possible to have '-' or minus quantity
    csv_data['quantity'] = remove_non_value.astype(str).astype(int)
     
    csv_branch.append(csv_data)
    
csv_branch_df = pd.concat(csv_branch)

In [5]:
# Read all relevant branch json files and add a new column called 'branch_name'.
# Clean and convert the 'quantity' field from an object to int.
json_branch = [] 
for i in globbed_json_files:

    json_data = pd.read_json(i)
    json_data.set_axis(headers, axis=1, inplace=True) # convert headers and specify axis
    extracted_filename = os.path.basename(i[15:-5]) # removes '20xx-20xx_' and '.json' from the filename
    remove_characters = extracted_filename.replace('_',' ') # remove '_' between the branch terms
    json_data['branch_name'] = remove_characters # 'branch_name' is consistent with other files

    remove_non_value = json_data['quantity'].replace('-','0') # remove any '-' from quantity field; not possible to have '-' or minus quantity
    json_data['quantity'] = remove_non_value.astype(str).astype(int)


    json_branch.append(json_data)
json_branch_df = pd.concat(json_branch)

In [6]:
# Join the csv and json branches.
branches = [csv_branch_df, json_branch_df]
all_branch_df = pd.concat(branches)

In [7]:
# Read the 'products_list.csv'.
products_list_df = pd.read_csv('products_list.csv')

In [8]:
# Merge all branches and product data, merging with product header.
all_branch_products_df = all_branch_df.merge(products_list_df, how='left')

In [9]:
# Read 'branch_list.xlsx' and 'branch_expenses.xlsx'.
region_df = pd.read_excel('data/branch_list.xlsx')
expenses_df = pd.read_excel('data/branch_expenses.xlsx')

# Merge region_df and expenses_df, merging with 'branch_name'.
region_and_expenses_df = region_df.merge(expenses_df, how='left')

In [10]:
# Merge all_branch_products_df, region_and_expenses_df, merging with 'branch_name'.
all_bra_prod_reg_exp_df = all_branch_products_df.merge(region_and_expenses_df, how='left')

In [11]:
# Total up 'quantity' for 'category' per 'region' / 'county'. 
grouped_cat = all_bra_prod_reg_exp_df.groupby(['region', 'county', 'category'])['quantity'].sum().sort_values(ascending=False).reset_index()

In [12]:
# Total up 'quantity' for 'product' per 'region' / 'county'.
grouped_prod = all_bra_prod_reg_exp_df.groupby(['region', 'county', 'product'])['quantity'].sum().sort_values(ascending=False).reset_index()

In [13]:
# Write grouped categories csv file, per region/county.
# Comment out once completed if page is required to be rerun.
prep_grouped_cat = etl.fromdataframe(grouped_cat)
cat_output = etl.tocsv(prep_grouped_cat, 'grouped_categories.csv')

In [14]:
# Write grouped products csv file, per region/county.
# Comment out once completed if page is required to be rerun.
prep_grouped_prod = etl.fromdataframe(grouped_prod)
prod_output = etl.tocsv(prep_grouped_prod, 'grouped_products.csv')

In [15]:
# Total up quantity and amount_in_gbp per branch over each date and time
grouped_branch_perf = all_bra_prod_reg_exp_df.groupby(['region', 'county', 'branch_name', 'year', 'month', 'day', 'hour'])['quantity','amount_in_gbp'].sum().sort_values(by=['quantity','amount_in_gbp'], ascending=False).reset_index()

C:\Users\Neal\AppData\Local\Temp/ipykernel_5188/220770401.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_branch_perf = all_bra_prod_reg_exp_df.groupby(['region', 'county', 'branch_name', 'year', 'month', 'day', 'hour'])['quantity','amount_in_gbp'].sum().sort_values(by=['quantity','amount_in_gbp'], ascending=False).reset_index()


In [16]:
# Write grouped branch performance csv file, per region/county.
# This file encompases both performance and hours at generic level, ready for filtering down for sales per branch per hour.
# Comment out once completed if page is required to be rerun.
prep_branch_perf = etl.fromdataframe(grouped_branch_perf)
branch_perf_output = etl.tocsv(prep_branch_perf, 'grouped_branch_performance.csv')

In [17]:
# Create performance column in branch performance data.
columns_list = ['quantity', 'amount_in_gbp']
grouped_branch_perf['performance'] = grouped_branch_perf[columns_list].sum(axis=1)

In [18]:
# Total up overall performance of each branch per region and county.
overall_branch_perf = grouped_branch_perf.groupby(['region', 'county', 'branch_name'])['performance'].sum().sort_values(ascending=False).reset_index()

In [19]:
# Write overall branch performance csv file, per region/county.
# Comment out once completed if page is required to be rerun.
prep_overall_branch_perf = etl.fromdataframe(overall_branch_perf)
branch_perf_output = etl.tocsv(prep_overall_branch_perf, 'overall_branch_performance.csv')

In [20]:
# Sum expenses and sales and group by branch_name
branch_sales_expenses = all_bra_prod_reg_exp_df.groupby('branch_name')['amount_in_gbp', 'operational_cost', 'staff_bonuses', 'misc_expenses', 'waste_cost'].sum().reset_index()

C:\Users\Neal\AppData\Local\Temp/ipykernel_5188/2842588614.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  branch_sales_expenses = all_bra_prod_reg_exp_df.groupby('branch_name')['amount_in_gbp', 'operational_cost', 'staff_bonuses', 'misc_expenses', 'waste_cost'].sum().reset_index()


In [21]:
# Create total expenses column
expenses_list = ['operational_cost', 'staff_bonuses', 'misc_expenses', 'waste_cost']
branch_sales_expenses['total_expenses'] = branch_sales_expenses[expenses_list].sum(axis=1)

# Create profit column 
branch_sales_expenses['profit'] = branch_sales_expenses['amount_in_gbp'] - branch_sales_expenses['total_expenses']

In [22]:
# Sort values by profit (max values at the top)
profit_per_branch = branch_sales_expenses.sort_values(by='profit', ascending=False)

In [23]:
# Write profitability csv file, per branch_name.
# Comment out once completed if page is required to be rerun.
prep_profit_per_branch = etl.fromdataframe(profit_per_branch)
profit_per_branch_output = etl.tocsv(prep_profit_per_branch, 'profit_per_branch.csv')

In [45]:
# Group regions to aid with dash region selection.
remove_duplicates = overall_branch_perf.drop_duplicates(subset=['region'])
region_list = remove_duplicates['region'].reset_index()

In [46]:
# Write region list csv file.
# Comment out once completed if page is required to be rerun.
prep_region_list = etl.fromdataframe(region_list)
region_list_output = etl.tocsv(prep_region_list, 'region_list.csv')

In [12]:
# Group branches to aid with dash branch selection.
remove_duplicates = region_df.drop_duplicates(subset=['branch_name']) 
branch_list = remove_duplicates['branch_name'].reset_index()

In [9]:
# Write branch list csv file.
# Comment out once completed if page is required to be rerun.
prep_branch_name_list = etl.fromdataframe(branch_list)
branch_name_list_output = etl.tocsv(prep_branch_name_list, 'branch_name_list.csv')

In [10]:
# Script Ends